In [2]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from typing import List, Tuple, Dict
from icecream import ic
import random

logging.basicConfig(level=logging.DEBUG)

In [ ]:
'''CITIES = pd.read_csv('./italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km'''


In [3]:
def load_multiple_countries(file_paths: Dict[str, str]) -> Dict[str, Tuple[pd.DataFrame, np.ndarray]]:
    data = {}
    for country, path in file_paths.items():
        cities = pd.read_csv(path, header=None, names=['name', 'lat', 'lon'])
        dist_matrix = np.zeros((len(cities), len(cities)))
        for c1, c2 in combinations(cities.itertuples(), 2):
            dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geodesic(
                (c1.lat, c1.lon), (c2.lat, c2.lon)
            ).km
        data[country] = (cities, dist_matrix)
    return data

In [4]:
file_paths = {
    'Italy': './italy.csv',
    'Russia': './russia.csv'
    
}
data = load_multiple_countries(file_paths)

In [5]:
# Function to calculate the path cost (total length)
def path_cost(country_name: str, path: List[int], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> float:
    cities, dist_matrix = data[country_name]
    num_cities = len(cities)
    return sum(dist_matrix[path[i], path[(i + 1) % num_cities]] for i in range(num_cities))

In [6]:
def tweak(country_name: str, path: List[int], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> List[int]:
    """ Performs 2-opt optimization on the given path. """
    improved = True
    while improved:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path)):
                if j - i == 1: # Avoid reversing two consecutive points
                    continue
                new_path = path[:]
                new_path[i:j] = reversed(path[i:j])  # Reverse the subsequence
                if path_cost(country_name, new_path, data) < path_cost(country_name, path, data):
                    path = new_path
                    improved = True
    return path


	

# Knock Knock Neighbor Algorithm (KKN)

In [12]:

def KKN(country_name: str, start_city: str, data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> List[int]:
    cities, dist_matrix = data[country_name]
    start_city_index = cities[cities['name'] == start_city].index[0]
    path = [start_city_index]
    visited = set(path)
    current_city = start_city_index

    while len(visited) < len(cities):
        nearest_distance = float('inf')

        # Find nearest city
        for city_index in range(len(cities)):
            if city_index not in visited:
                distance = dist_matrix[current_city, city_index]
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_city = city_index

        path.append(nearest_city)
        visited.add(nearest_city)
        current_city = nearest_city

    path.append(start_city_index)  # Return to start city

    # Optimize path with 2-opt
    path = tweak( country_name,path, data)
    

    cost = path_cost(country_name, path, data)
    return path, cost



In [21]:
for country_name, (cities, dist_matrix) in data.items():
    start_city = cities['name'].iloc[random.randint(0, len(cities) - 1)]
    path, cost = KKN(country_name, start_city, data)
    #print the path (names of cities) and the cost of the path
    print(f'{country_name}: {path_cost(country_name, path, data):.2f} km')
    print(" Best path :", [cities['name'][i] for i in path])


Italy: 4309.30 km
 Best path : ['Milan', 'Monza', 'Bergamo', 'Brescia', 'Piacenza', 'Parma', "Reggio nell'Emilia", 'Modena', 'Bologna', 'Ferrara', 'Venice', 'Padua', 'Vicenza', 'Verona', 'Trento', 'Bolzano', 'Trieste', 'Ancona', 'Pescara', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calabria', 'Syracuse', 'Catania', 'Palermo', 'Cagliari', 'Sassari', 'Latina', 'Rome', 'Terni', 'Perugia', 'Rimini', 'Ravenna', 'Forlì', 'Florence', 'Prato', 'Leghorn', 'Genoa', 'Turin', 'Novara', 'Milan']
Russia: 34865.02 km
 Best path : ['Voronezh', 'Staryy Oskol', 'Belgorod', 'Taganrog', 'Krasnodar', 'Novorossiysk', 'Sochi', 'Maykop', 'Armavir', 'Stavropol', 'Nevinnomyssk', 'Cherkessk', 'Kislovodsk', 'Pyatigorsk', 'Nalchik', 'Vladikavkaz', 'Nazran', 'Groznyy', 'Khasavyurt', 'Makhachkala', 'Kaspiysk', 'Derbent', 'Astrakhan', 'Elista', 'Bataysk', 'Rostov‐na‐Donu', 'Novocherkassk', 'Novoshakhtinsk', 'Shakhty', 'Volgodonsk', 'Volgograd', 'Volzhski